In [8]:
# 내가 생각하는 최종ver.불용어 제거(이모티콘 축약)
import pandas as pd
import stopwordsiso as stopwords
from konlpy.tag import Okt
import re
from konlpy.tag import Okt


# 1) CSV 로드
df = pd.read_csv('/Users/leejuan/Documents/GitHub/webnovel-crawler/data/raw/[알수없는회차]_댓글_20250617_181947.csv')

# 2) 불용어 집합 정의
base_stopwords = stopwords.stopwords("ko")
domain_stopwords = {
    "화수", "회차",
    "미리보기", "무료",
    "<div>", "</div>", "<br>", "&nbsp;",
    "href", "class", "id", "onclick",
    "댓글", "작성", "작성자", "조회", "추천", "공감",
    "작가님", "작가", "작품", "내용", "부분"
}
custom_stopwords = base_stopwords.union(domain_stopwords)

# 3) 이모티콘 패턴 정의 (연속된 동일 자모가 이모티콘)
emoticon_pattern = re.compile(r'^(?:[ㅋㅎㅠㅜ])+$')

# 4) 이모티콘 정규화 함수: 중복된 문자를 한 글자로 축소
def normalize_emoticon(tok: str) -> str:
    # 반복된 ㅋ, ㅎ, ㅠ, ㅜ 글자 중 첫 글자만 남김
    return re.sub(r'([ㅋㅎㅠㅜ])\1+', r'\1', tok)

def is_emoticon(tok: str) -> bool:
    return bool(emoticon_pattern.fullmatch(tok))

# 5) 형태소 분석기 초기화
okt = Okt()

# 6) 전처리 함수
def preprocess(text: str) -> str:
    if not isinstance(text, str):
        return ""
    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', ' ', text)
    # 형태소 단위 토큰화 및 어간 추출
    tokens = okt.morphs(text, stem=True)

    filtered = []
    for tok in tokens:
        # 이모티콘은 정규화하여 보존
        if is_emoticon(tok):
            filtered.append(normalize_emoticon(tok))
            continue
        # 불용어 제거
        if tok in custom_stopwords:
            continue
        # 너무 짧은 토큰 제거
        if len(tok) <= 1:
            continue
        filtered.append(tok)

    return " ".join(filtered)

# 7) df['댓글'] 컬럼에 적용
df['댓글'] = df['댓글'].fillna('').apply(preprocess)

def extract_nav(text: str) -> str:
    # norm=True: 발화체 정규화, stem=True: 어간 추출
    pos_tags = okt.pos(text, norm=True, stem=True)
    # 명사, 형용사, 동사 품사만 필터링
    filtered = [word for word, pos in pos_tags if pos in ('Noun', 'Adjective', 'Verb')]
    # (선택) 너무 짧은 토큰 제거
    filtered = [tok for tok in filtered if len(tok) > 1]
    return " ".join(filtered)

# 새로운 컬럼에 적용
df['댓글'] = df['댓글'].apply(extract_nav)


# 8) 결과 확인
print(df[['댓글']].head())

# 9) (선택) 전처리된 데이터 저장 (processed_nahonja_levelup_cleaned.csv)
df.to_csv('/Users/leejuan/Documents/GitHub/webnovel-crawler/data/processed/kim_processed_1.csv', index=False)

                                                  댓글
0                                          비서 그렇다 장판
1                           회장 그냥 미소 며느리 삼고 싶다 말씀 하다
2  동생 관심 쏠리다 질투 하다 동생 벌다 진일 벌어지다 일로 착각 하다 된거 동생 경...
3                         배다 웃기 죽겠네 그동안 보지 하다 캐릭터 남다
4                           원래 오빠 하다 괜찮다 오빠 하다 꼴다 보다


In [10]:
import os
import pandas as pd
from konlpy.tag import Okt
import re

# 1. 사전 불러오기 함수
def load_word_list(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return set(line.strip() for line in f if line.strip())


# 2. 경로 설정

pos_path = '/Users/leejuan/Documents/GitHub/webnovel-crawler/scripts/pos_words.txt'
neg_path = '/Users/leejuan/Documents/GitHub/webnovel-crawler/scripts/neg_words.txt'
pos_words = load_word_list(pos_path)
neg_words = load_word_list(neg_path)


# 3. 형태소 분석기 초기화
okt = Okt()


# 4. 텍스트 전처리 함수
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'<[^>]+>', ' ', text)         # HTML 태그 제거
    text = re.sub(r'[^가-힣\s]', ' ', text)       # 특수문자 제거
    return text


# 5. 감정 분석 함수
def rule_sentiment(text: str) -> str:
    text = clean_text(text)
    tokens = okt.morphs(text, stem=True)

    pos_score = sum(1 for t in tokens if t in pos_words)
    neg_score = sum(1 for t in tokens if t in neg_words)

    if pos_score > neg_score:
        return "Positive"
    elif neg_score > pos_score:
        return "Negative"
    else:
        return "Neutral"


# 6. CSV 불러오기 및 분석 적용
df = pd.read_csv(os.path.join(base_dir, 'data', 'processed', 'kim_processed_1.csv'))

df['감정_라벨'] = df['댓글'].apply(rule_sentiment)


# 7. 결과 저장
base_dir = '/Users/leejuan/Documents/GitHub/webnovel-crawler'
df.to_csv(os.path.join(base_dir, 'data', 'processed', 'kim_processed_final.csv'), index=False, encoding='utf-8-sig')


# 8. 통계 확인

print(df[['댓글', '감정_라벨']].head())

NameError: name 'base_dir' is not defined

In [72]:
df['감정_라벨'].value_counts() 

감정_라벨
Neutral     4441
Positive    3759
Negative    1801
Name: count, dtype: int64